In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [2]:
from deepface import DeepFace
import os
import pandas as pd

2023-09-27 11:44:20.182246: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-27 11:44:20.184438: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-27 11:44:20.231501: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-27 11:44:20.232232: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 11:44:21.040994: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
img_dir = "/var/wd_smit/internal_repos/backups/images"
models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "Dlib", "ArcFace"]
embeddings = []
face_coords = []
names = []

df = pd.DataFrame({"embedding": [], "facial_area": []})

for img_name in os.listdir(img_dir):
    img_path = os.path.join(img_dir, img_name)
    embedding = DeepFace.represent(img_path, model_name=models[1])
    embeddings.append(embedding[0]["embedding"])
    face_coords.append(embedding[0]["facial_area"])
    names.append(img_name)

    dftemp = pd.DataFrame(embedding)
    df = pd.concat([df, dftemp], ignore_index=True)

df["name"] = names

# df['embedding'] = df['embedding'].apply(lambda x: ','.join(map(str, x)))
# df['facial_area'] = df['facial_area'].apply(lambda x: ','.join(map(str, x)))

2023-09-27 11:44:21.920549: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 11:44:21.920877: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 11:44:21.921164: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
len(df['embedding'][2])

128

In [5]:
type(df['embedding'][2])

list

In [6]:
df['embedding'][2]

[-0.8615654706954956,
 -0.691304087638855,
 -0.02600961923599243,
 -0.33501437306404114,
 0.04195718467235565,
 0.11102785915136337,
 0.016428720206022263,
 0.15121285617351532,
 -0.09640228748321533,
 0.010267660021781921,
 0.1332949995994568,
 0.49596381187438965,
 0.4241420030593872,
 -0.42292293906211853,
 0.007298417389392853,
 -0.5913308262825012,
 0.3133944272994995,
 0.15267929434776306,
 0.2282409816980362,
 -0.5085062980651855,
 0.029473260045051575,
 -0.5036550164222717,
 0.040847212076187134,
 -0.004681050777435303,
 0.21379540860652924,
 -0.31052738428115845,
 0.6320617198944092,
 -0.13005109131336212,
 -0.4827842116355896,
 -0.4682225286960602,
 -0.7054212093353271,
 0.16479316353797913,
 -0.2541625201702118,
 0.4381856918334961,
 0.5066251754760742,
 0.06255875527858734,
 0.6990558505058289,
 -0.40486565232276917,
 -0.025259539484977722,
 -0.5732600688934326,
 0.48674696683883667,
 0.9974526762962341,
 0.28450530767440796,
 0.5473789572715759,
 -0.1301756352186203,
 -0.7

In [7]:
df['facial_area']

0        {'x': 97, 'y': 26, 'w': 82, 'h': 82}
1       {'x': 102, 'y': 30, 'w': 79, 'h': 79}
2    {'x': 150, 'y': 176, 'w': 519, 'h': 519}
3        {'x': 38, 'y': 56, 'w': 96, 'h': 96}
Name: facial_area, dtype: object

In [8]:
templist = []

for i in range(len(df['facial_area'])):
    temp = []
    for key, value in df['facial_area'][i].items():
        temp.append(value)
    templist.append(temp)

In [9]:
templist

[[97, 26, 82, 82], [102, 30, 79, 79], [150, 176, 519, 519], [38, 56, 96, 96]]

In [10]:
df['facial_area'] = templist

In [11]:
df['facial_area']

0        [97, 26, 82, 82]
1       [102, 30, 79, 79]
2    [150, 176, 519, 519]
3        [38, 56, 96, 96]
Name: facial_area, dtype: object

In [12]:
df

,embedding,facial_area,name
0,"[-0.40423446893692017, -0.031015239655971527, ...","[97, 26, 82, 82]",Untitled.jpg
1,"[-1.147232174873352, 0.5872104167938232, 0.777...","[102, 30, 79, 79]",Untitled2.jpg
2,"[-0.8615654706954956, -0.691304087638855, -0.0...","[150, 176, 519, 519]",Elon_Musk_Colorado_2022_(cropped2).jpg
3,"[-1.1388880014419556, 0.7310553789138794, -1.6...","[38, 56, 96, 96]",Untitled1.jpg


In [13]:
# # Create an index
# es.indices.create(index='base_index', body={
#   "mappings": {
#     "properties": {
#       "embedding": {
#         "type": "dense_vector",
#         "dims": 128
#       },
#       "facial_area": {
#         "type": "dense_vector",
#         "dims": 4
#       },
#       "name":{
#         "type": "keyword"
#       }
#     }
#   }
# })

In [14]:
# for i, row in df.iterrows():
#     # Convert the row to a dictionary
#     doc = row.to_dict()

#     # Index the document
#     es.index(index='base_index', body=doc)

In [15]:
# queryvec = df['embedding'][2]

In [16]:
# res = es.search(index='my-index', body={
#     'query': {
#         'match': {
#             'embedding': queryvec
#         }
#     }
# })

In [17]:
# res

In [18]:
# res['hits']

In [19]:
# resp = es.search(index="base_index", query={"match_all": {}})

In [20]:
# resp

In [21]:
# es.indices.delete(index='*')

In [22]:
%pip install scipy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [23]:
import numpy as np
from scipy import spatial

In [24]:
list_of_distances = []

for i in df['embedding']:
    temp = []
    for j in df['embedding']:
        result = 1 - spatial.distance.cosine(i, j)
        temp.append(result)
    list_of_distances.append(temp)    

In [25]:
list_of_distances

[[1, 0.03148167553714143, 0.07794618569688394, 0.19068504593729085],
 [0.03148167553714143, 1, -0.11457911017833777, 0.23277863016311273],
 [0.07794618569688394, -0.11457911017833777, 1, 0.0733383233230952],
 [0.19068504593729085, 0.23277863016311273, 0.0733383233230952, 1]]

In [27]:
stats = []

In [26]:
for i in list_of_distances:
    print(i)

[1, 0.03148167553714143, 0.07794618569688394, 0.19068504593729085]
[0.03148167553714143, 1, -0.11457911017833777, 0.23277863016311273]
[0.07794618569688394, -0.11457911017833777, 1, 0.0733383233230952]
[0.19068504593729085, 0.23277863016311273, 0.0733383233230952, 1]


In [28]:
for i in list_of_distances:
    temp = []
    maximum = 0
    for j in i:
        if j != 1:
            if j > maximum:
                maximum = j

    print(maximum, i.index(maximum))
    temp.append(maximum)
    temp.append(i.index(maximum))    
    stats.append(temp)

0.19068504593729085 3
0.23277863016311273 3
0.07794618569688394 0
0.23277863016311273 1


In [29]:
stats

[[0.19068504593729085, 3],
 [0.23277863016311273, 3],
 [0.07794618569688394, 0],
 [0.23277863016311273, 1]]